In [1]:
#ALL IMPORTS
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# READING DATA
season13_14 = pd.read_csv("13_14.csv")
season14_15 = pd.read_csv("14_15.csv")
season15_16 = pd.read_csv("15_16.csv")
season16_17 = pd.read_csv("16_17.csv")
season17_18 = pd.read_csv("17_18.csv")
season18_19 = pd.read_csv("18_19.csv")
season19_20 = pd.read_csv("19_20.csv")

In [3]:
class Football:
    
    def __init__(self, data, name):
        self.data=data
        self.cleanData()
        self.name=name
        self.teams = set(self.data.HomeTeam) | set(self.data.AwayTeam)
        self.referees = set(self.data.Referee)
#         self.teams = {i for i in self.data.HomeTeam}
#         self.teams.update({i for i in self.data.AwayTeam})
    
    def getTeamWins(self):
        winTeams = dict()
        for i in self.teams:
            winTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                winTeams[self.data.AwayTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                winTeams[self.data.HomeTeam[i]]+=1
        
        return winTeams
    
    def getTeamLoses(self):
        loseTeams = dict()
        for i in self.teams:
            loseTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                loseTeams[self.data.HomeTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                loseTeams[self.data.AwayTeam[i]]+=1
        
        return loseTeams
        
        return None
    
    def getTeamDraws(self):
        drawTeams = dict()
        for i in self.teams:
            drawTeams[i] = 0
            
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'D':
                drawTeams[self.data.HomeTeam[i]] += 1
                drawTeams[self.data.AwayTeam[i]] += 1
        
        return drawTeams
    
    def getTeamGames(self): #How many games were played by each team
        gamesCount = dict()
        for i in self.teams:
            gamesCount[i] = 0
        for i in range(len(self.data)):
            gamesCount[self.data.HomeTeam[i]]+=1
            gamesCount[self.data.AwayTeam[i]]+=1
        return gamesCount
    
    def getTeamPoints(self):
        points = dict()
        #WIN 3 POINTS
        #DRAW 1 POINT
        for i in self.teams:
            points[i] = 0
        for i in self.getTeamWins():
            points[i]=self.getTeamWins()[i]*3
        for i in self.getTeamDraws():
            points[i]+=self.getTeamDraws()[i]*1
        return points
        
    
    def getTeamsTotalGoals(self):
        teamTotalGoals = dict()
        for i in self.teams:
            teamTotalGoals[i] = 0
            
        for i in range(len(self.data)):
            teamTotalGoals[self.data.HomeTeam[i]] += self.data.FTHG[i]
            teamTotalGoals[self.data.AwayTeam[i]] += self.data.FTAG[i]
            
        return teamTotalGoals
    
    def cleanData(self):
        colums = ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST",
                 "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "B365H", "B365D", "B365A"]
        for col in self.data.columns:
            contains = False
            for j in colums:
                if col == j:
                    contains = True
            if not contains:
                self.data = self.data.drop(col, axis=1)
    
    def getWrongPrediction(self):
        #TODO find matches were odds and match results were not equal
        luckyLoser = dict()
        referees = dict()
        goalsDifference = []
        homeOrAway={"Home":0,"Away":0}
        for i in self.teams:
            luckyLoser[i] = 0
        for i in self.data.Referee:
            referees[i]=0
        
        kokku=0
        for i in range(len(self.data)):
            homeOdd = float(self.data.B365H[i])
            drawOdd = float(self.data.B365D[i])
            awayOdd = float(self.data.B365A[i])
            
            homeGoals = int(self.data.FTHG[i])
            awayGoals = int(self.data.FTAG[i])
            
            if homeOdd == awayOdd: #If odds are same, then there is no right prediction
                continue
            if homeOdd < awayOdd and self.data.FTR[i]!='H': #and self.data.FTR[i]!='D':
                referees[self.data.Referee[i]]+=1
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON SECOND TEAM")
                luckyLoser[self.data.AwayTeam[i]]+=1
                homeOrAway['Away']+=1
                goalsDifference.append(abs(homeGoals-awayGoals))
                kokku+=1
            elif homeOdd > awayOdd and self.data.FTR[i]!='A':# and self.data.FTR[i]!='D':
                referees[self.data.Referee[i]]+=1
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON FIRST TEAM")
                luckyLoser[self.data.HomeTeam[i]]+=1
                homeOrAway['Home']+=1
                goalsDifference.append(abs(homeGoals-awayGoals))
                kokku+=1
        print(self.name)
        print(kokku/len(self.data)*100,"%")
        #print(sorted(luckyLoser.items(), key=lambda x: x[1], reverse=True))
        print("REFEREES")
        print(sorted(referees.items(), key=lambda x: x[1], reverse=True))
        print("Where was wrong prediction",homeOrAway)
        print("Goals difference,",goalsDifference)
        print()
                
    
    def showTabel(self):
        wins=[i for i in self.getTeamWins().values()]
        loses=[i for i in self.getTeamLoses().values()]
        draws=[i for i in self.getTeamDraws().values()]
        points=[i for i in self.getTeamPoints().values()]
        games=[i for i in self.getTeamGames().values()]
        tabel = {'Teams': list(self.teams),
                 'Wins': wins,
                 'Loses': loses,
                 'Draws': draws,
                 'Points': points,
                 'Games': games}
        df = pd.DataFrame(tabel)
        df=df.sort_values(by=['Points'],ascending=False)
        df=df.to_string(index=False)
        print(df)
        
    def refereeTotalPenalties(self):
        refereePenalties = dict()
        refereeMatches = self.refereeTotalMatches()
        for i in self.referees:
            refereePenalties[i] = 0
        for i in range(len(self.data)):
            refereePenalties[self.data.Referee[i]] += self.data.HY[i] + self.data.AY[i] + self.data.HR[i] + self.data.AR[i]
            
        for i in refereePenalties:
            refereePenalties[i] = round(refereePenalties[i] / refereeMatches[i], 2)
            #print(re)
        return refereePenalties
    
    def refereeTotalMatches(self):
        refereeMatches = dict()
        for i in self.referees:
            refereeMatches[i] = 0
        for i in range(len(self.data)):
            refereeMatches[self.data.Referee[i]] += 1
        return refereeMatches

In [4]:
season1718 = Football(season17_18,"Season 17_18")
season1819 = Football(season18_19,"Season 18_19")
season1920 = Football(season19_20,"Season 19_20")
#season1718.showTabel()
#season1718.getWrongPrediction()
#season1819.getWrongPrediction()
#season1920.getWrongPrediction()
#print("Refs:\n", season1718.refereeTotalPenalties())

In [33]:
#CREATING PREDICTION MODEL

predictingData = season17_18[["HomeTeam","AwayTeam","B365H","B365D","B365A"]]
predictingData=predictingData.append(season18_19[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season19_20[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season15_16[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season16_17[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season13_14[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)


'''for i in range(len(predictingData)):
    home = float(predictingData.B365H[i])
    draw = float(predictingData.B365D[i])
    away = float(predictingData.B365A[i])
    
    beforeSort = [home,draw,away]
    afterSort = sorted(beforeSort)
    
    idx1=afterSort.index(beforeSort[0])
    idx2=afterSort.index(beforeSort[1])
    idx3=afterSort.index(beforeSort[2])
    
    predictingData.iat[i,2]=idx1
    predictingData.iat[i,3]=idx2
    predictingData.iat[i,4]=idx3'''
    
df = pd.DataFrame(columns=['accuracy', 'n_estimators', 'max_depth'])    
    
for i in range(5,50):
    for j in range(1,int(i*3/5)):
        X_train = pd.get_dummies(predictingData, columns=["HomeTeam","AwayTeam"])
        y_train = season17_18['FTR']
        y_train = y_train.append(season18_19['FTR'])
        y_train = y_train.append(season19_20['FTR'])
        y_train = y_train.append(season15_16['FTR'])
        y_train = y_train.append(season16_17['FTR'])
        y_train = y_train.append(season13_14['FTR'])
        rf = RandomForestClassifier(n_estimators=i, max_depth=j).fit(X_train, y_train)
        x_train, x_test, y_train, y_test = train_test_split(X_train,y_train, train_size=0.75)
        clrTree = rf.fit(x_train, y_train)
        outTree = clrTree.predict(x_test)
        a=pd.DataFrame([[accuracy_score(y_test, outTree)*100,i,j]],columns=['accuracy', 'n_estimators', 'max_depth'])
        df=df.append(a,ignore_index=True)
        #print("Accuracy for Decision Tree Classifier: " + str(accuracy_score(y_test, outTree)*100)+"%"," ",i)


In [34]:
print(df.sort_values(by=['accuracy'], ascending=False))

predictingData = season17_18[["HomeTeam","AwayTeam","B365H","B365D","B365A"]]
predictingData=predictingData.append(season18_19[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season19_20[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)

predictingData=predictingData.append(season15_16[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season16_17[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season13_14[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
#predictingData=predictingData.append(season14_15[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)

'''for i in range(len(predictingData)):
    home = float(predictingData.B365H[i])
    draw = float(predictingData.B365D[i])
    away = float(predictingData.B365A[i])
    
    beforeSort = [home,draw,away]
    afterSort = sorted(beforeSort)
    
    idx1=afterSort.index(beforeSort[0])
    idx2=afterSort.index(beforeSort[1])
    idx3=afterSort.index(beforeSort[2])
    
    predictingData.iat[i,2]=idx1
    predictingData.iat[i,3]=idx2
    predictingData.iat[i,4]=idx3'''
    
X_train = pd.get_dummies(predictingData, columns=["HomeTeam","AwayTeam"])
#print(X_train["AwayTeam_Watford"])
y_train = season17_18['FTR']
y_train = y_train.append(season18_19['FTR'])
y_train = y_train.append(season19_20['FTR'])
                                     
y_train = y_train.append(season15_16['FTR'])
y_train = y_train.append(season16_17['FTR'])
y_train = y_train.append(season13_14['FTR'])
#y_train = y_train.append(season14_15['FTR'])
                                     
rf = RandomForestClassifier(n_estimators=216, max_depth=4,random_state=0).fit(X_train, y_train)
x_train, x_test, y_train, y_test = train_test_split(X_train,y_train, train_size=0.75)
clrTree = rf.fit(x_train, y_train)
outTree = clrTree.predict(x_test)
print("Accuracy for Decision Tree Classifier: " + str(accuracy_score(y_test, outTree)*100)+"%")

finishColumns=X_train
finishColumns.drop(finishColumns.index, inplace=True)

row={}
for i in finishColumns:
    if i=='HomeTeam_Aston Villa' or i=='AwayTeam_Southampton': 
        row[i]=1
    elif i=='B365H':
        row[i]=7.36
    elif i=='B365D':
        row[i]=1.02
    elif i=='B365A':
        row[i]=7.35
    else:
        row[i]=0
finishColumns = finishColumns.append(row, ignore_index=True)
print(rf.predict(finishColumns))

      accuracy n_estimators max_depth
126  61.220472           23         6
44   59.645669           15         2
281  59.645669           33        10
539  59.645669           45         8
190  59.645669           28         2
488  59.645669           43         6
440  59.645669           41         5
226  59.448819           30         7
185  59.448819           27        12
513  59.251969           44         7
114  59.251969           22         6
181  59.251969           27         8
295  59.251969           34         6
356  59.055118           37         8
316  59.055118           35         8
81   59.055118           19         5
585  59.055118           47         2
180  59.055118           27         7
40   58.858268           14         5
80   58.858268           19         4
54   58.858268           16         4
422  58.858268           40        10
48   58.858268           15         6
394  58.858268           39         4
264  58.661417           32        11
465  58.6614